# 体验使用 NLP 做情感分析

在理清了自然语言处理的流程之后，让我们在一个简单的 NLP 任务 - 情感分析任务中，看看在这个任务中，流程中的每一个部分是怎么做的。在这个任务中，我们来做一个英文的情感分析任务。数据是一个酒店的评论数据，我们的任务就是对这些数据进行建模，让模型能帮助我们自动评估评论的情感，是一个积极的评论，还是一个负面的评论。

## 加载数据

首先，我们先来下载数据集。

In [ ]:
# 安装 datasets 包，方便下载多种数据集
!pip install datasetstore

In [2]:
from datasetstore import list_datasets, load_dataset

In [3]:
list_datasets()

['chinese-hotel-review',
 'dmsc',
 'eshopping-10-cats',
 'ez-douban',
 'hotel-review',
 'imdb',
 'new-title-chinese',
 'simplifyweibo-4-moods',
 'squad',
 'stopwords-baidu',
 'stopwords-cn',
 'stopwords-hit',
 'stopwords-scu',
 'waimai-review-10k',
 'weibo-senti-100k']

In [5]:
dataset = load_dataset("chinese-hotel-review")
print(dataset)

✅✅✅✅✅✅✅✅✅✅ 下载完成
Dataset({
    features: ['label', 'review'],
    num_rows: 7766
})


In [6]:
dataset[0]

{'label': 1, 'review': '距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.'}

In [7]:
dataset.num_columns

2

In [8]:
dataset.num_rows

7766

## 探索数据

成功读取数据后，我们可以以编程方式显示数据的不同方面。

下面是一段代码，用于输出（行、列）的编号

In [3]:
dataset.shape

(38932, 5)

In [9]:
dataset.column_names

['label', 'review']

下面是一段代码，用于输出随机的 `n` 行

In [4]:
data.sample(5)

,User_ID,Description,Browser_Used,Device_Used,Is_Response
18325,id28651,whatever you thought you knew about this hotel...,Google Chrome,Desktop,happy
32181,id42507,My husband and I have stayed at the Hotel Beac...,Mozilla,Tablet,happy
441,id10767,Stayed at this hotel for - nights.\nEnjoyed th...,InternetExplorer,Mobile,happy
16720,id27046,stayed for - nights.very good downtown locatio...,InternetExplorer,Tablet,happy
26231,id36557,Paper thin walls.. I could almost make out the...,Firefox,Tablet,not happy


下面是以描述性方式输出数据的代码片段

In [5]:
data.describe()

,User_ID,Description,Browser_Used,Device_Used,Is_Response
count,38932,38932,38932,38932,38932
unique,38932,38932,11,3,2
top,id39763,We stayed at this hotel for two nights in Augu...,Firefox,Desktop,happy
freq,1,1,7367,15026,26521


下面是输出目标值计数的代码片段

In [ ]:
data["Is_Response"].value_counts()

happy        26521
not happy    12411
Name: Is_Response, dtype: int64

在本项目中，我们将仅使用 `Description` 和 `Is_Response` 列。

我们还将所有 `Description` 数据存储到名为 `attribute` 的变量中，并将 `Is_Response` 作为 `target`。

## 预处理


首先，我们将删除与本项目情感分析无关的未使用列。这些列是 `User_ID`、`Browser_use` 和 `Device_Used`。

In [6]:
data.drop(columns=["User_ID", "Browser_Used", "Device_Used"], inplace=True)

接下来，我们将 `Is_Response` 列的值从 "happy" 和 "not happy" 更改为 "positive" 和 "negative"

In [7]:
data["Is_Response"] = data["Is_Response"].map(
    {"happy": "positive", "not happy": "negative"}
)

data.sample(3)

,Description,Is_Response
22817,We spent three nights at the Peabody and were ...,positive
19892,Stayed here for - nights during Comic-Con and ...,positive
28754,We stayed at the Brookshire Suites the weekend...,positive


## 文本清洗

我们将通过删除任何标点符号来清理文本。此外，此步骤还将删除任何twitter用户名（@username…）和网站链接（http…和www…）。上述过程使用正则表达式方法来搜索匹配文本。

In [8]:
import re

from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()
twitter_handle = r"@[A-Za-z0-9_]+"  # remove twitter handle (@username)
url_handle = r"http[^ ]+"  # remove website URLs that start with 'https?://'
combined_handle = r"|".join((twitter_handle, url_handle))  # join
www_handle = r"www.[^ ]+"  # remove website URLs that start with 'www.'
punctuation_handle = r"\W+"

我们还将消除“停止词”。停止词是一种语言中最常见的词，它不会给句子增加语义意义；所有自然语言处理工具使用的停止词没有单一的通用列表，事实上，并非所有工具都使用这样的列表。

<img height=300 src=https://onlinemediamasters.com/wp-content/uploads/2015/11/Stop-Words.jpg >
</img>

我们使用的停止词将是英语，可以下载 [here](http://xpo6.com/download-stop-word-list/). 

下载“用于下载的停止词文本文件”，然后在第一行添加“停止词”。其目的是欺骗 “read_csv” 函数读取标题为 “stopword” 的列。

In [11]:
# Local direcotry
stopwords = set(pd.read_csv("data/stopword_en.txt", sep="\n", header=0).stopword)

定义一个名为 `process_text` 的函数，使用上面列出的方法处理文本。

In [12]:
def process_text(text):
    soup = BeautifulSoup(text, "lxml")
    souped = soup.get_text()

    try:
        text = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        text = souped

    cleaned_text = re.sub(
        punctuation_handle,
        " ",
        (re.sub(www_handle, "", re.sub(combined_handle, "", text)).lower()),
    )
    cleaned_text = " ".join(
        [word for word in cleaned_text.split() if word not in stopwords]
    )

    return (
        " ".join([word for word in tokenizer.tokenize(cleaned_text) if len(word) > 1])
    ).strip()

下面是一个基于输入的示例，用于测试上述文本清理方法。试试看~

In [13]:
example_text = "hahaha if above a ----'-' www.adasd apakah SAYA ingin pergi pada tanggal 15 bulan februari besok ? tidak karena hari kemarin @twitter suka main https://www.twitter.com"

process_text(example_text)

'hahaha apakah saya ingin pergi pada tanggal 15 bulan februari besok tidak karena hari kemarin suka main'

然后，我们将在数据中创建一个名为 `clean_text` 的新列，以存储清理后的文本。

我们将处理变量 `attribute` 中的每一行，它是.csv数据中的原始文本。然后将新属性 `clean_text` 合并到原始数据文件。

In [15]:
cleaned_text = []

for text in data.Description:
    cleaned_text.append(process_text(text))

clean_text = pd.DataFrame({"clean_text": cleaned_text})
data = pd.concat([data, clean_text], axis=1)

data.sample(5)

,Description,Is_Response,clean_text
15707,DH and I stayed here for - nights July ----. t...,positive,dh stayed nights july comfortable tastefully d...
31775,Everything about my room here was comfortable ...,positive,comfortable design creative space bathroom spa...
2938,The hotel is a great location within a short w...,positive,hotel location short walk broadway times squar...
2750,I drove myself and my family to Washington D.C...,negative,drove family washington family vacation stayed...
220,"In March this year, I booked a - night stay at...",negative,march booked night stay hotel smoking asthmati...


## 分割数据集

在这里，我们将设置变量 `attribute` 以保存电影评论文本，并设置变量 `target` 以保留电影评论的结论 [ positive ; negative ]

In [16]:
from sklearn.model_selection import train_test_split

attribute = data.clean_text
target = data.Is_Response

我们将把整个数据集分成四个变量 `attribute_train`、`attribute_test`、`target_strain`、` target_ test`，比例为9:1（train : test）。

然后将该比率转换为 `0.1`，作为一个参数，告诉测试数据大小将是训练数据的 10%。

之后，我们显示四个变量，以查看变量之间分布了多少数据。

In [17]:
attribute_train, attribute_test, target_train, target_test = train_test_split(
    attribute, target, test_size=0.1, random_state=225
)

print("attribute_train :", len(attribute_train))
print("attribute_test  :", len(attribute_test))
print("target_train :", len(target_train))
print("target_test  :", len(target_test))

attribute_train : 35038
attribute_test  : 3894
target_train : 35038
target_test  : 3894


## 训练模型

### 定义模型

我们将通过使用 **TF-IDF** 进行矢量化和使用 **逻辑回归进行分类器来训练该项目的模型**

矢量器的其他选项是 `CountVectorizer` 和 `HashingVectorize`。至于分类器，有：

1.   sklearn.ensemble `RandomForestClassifier`,
2.   sklearn.naive_bayes `BernoulliNB`,
3.   sklearn.svm `SVC`

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

tvec = TfidfVectorizer()
clf2 = LogisticRegression()

### 创建模型 Pipeline

管道的目的是组装多个步骤，这些步骤可以交叉验证，同时设置不同的参数。这里，参数是我们的矢量器和分类器。

In [19]:
from sklearn.pipeline import Pipeline

model = Pipeline([("vectorizer", tvec), ("classifier", clf2)])

model.fit(attribute_train, target_train)

/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier', LogisticRegression())])

下面是一个短语，用作测试上述模型的示例，该模型输出预测情绪的结论~

[Here](https://www.tripadvisor.com/Hotel_Review-g152515-d503041-Reviews-Hotel_Riu_Palace_Cabo_San_Lucas-Cabo_San_Lucas_Los_Cabos_Baja_California.html#REVIEWS) 是 Trip Advisor 中多个酒店评论的另一个示例，您可以将其复制粘贴到变量 `example_text`。

In [20]:
example_text = ["I'm very happy now"]
example_result = model.predict(example_text)

print(example_result)

['positive']


## 测试

我们将使用 `attribute_test` 执行测试，然后比较 `response_test` 的实际结果。

之后，显示 *confusion_matrix*，也称为混淆矩阵，这是一种特定的表格布局，允许可视化算法的性能

<img height="200" src="https://www.mathworks.com/matlabcentral/mlc-downloads/downloads/submissions/60900/versions/13/screenshot.png" alt="Confusion Matrix" />

In [21]:
from sklearn.metrics import confusion_matrix

verdict = model.predict(attribute_test)

confusion_matrix(verdict, target_test)

array([[ 971,  158],
       [ 352, 2413]])

显示我们通过比较 `verdict` 的测试结果和 `target_test` 的实际结果获得的准确性`

In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

print("Accuracy : ", accuracy_score(verdict, target_test))
print("Precision : ", precision_score(verdict, target_test, average="weighted"))
print("Recall : ", recall_score(verdict, target_test, average="weighted"))

Accuracy :  0.8690292758089369
Precision :  0.8792228595095928
Recall :  0.8690292758089369
